<h1 style="font-size: 50px;">Research Project 2 Epilepsy Ionic Modulation - SmallBrain</h1>

<br></br><br></br>

<h2 style="font-size: 40px;">Installing and Importing Libraries</h2>

In [5]:
# Installing all the required libraries.
!pip install -q jupyter
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q plotly
!pip install -q scipy
!pip install -q tqdm
!pip install -q Brian2

In [6]:
from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

In [7]:
# Importing all the required libraries.
import os
from brian2 import *
import random
from tqdm.notebook import tqdm
import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')

<br></br>

<h2 style="font-size: 40px;">General Functions</h2>

In [8]:
# Reads the stimulus input signal.
def read_input_signal(file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    # Converting the signal to be of type 'TimedArray' with a specified time step which can be used in the simulation.
    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal

<br></br>

<h2 style="font-size: 40px;">Topology and Neuron Groups Functions</h2>

In [9]:
# Creates the 3D space of neurons (0.6 mm x 0.6 mm x 0.6 mm).
def create_neuron_topology(N, bounds):
    
    x_bound, y_bound, z_bound = bounds

    # Creates a set of random x, y, and z points, in total N points, that fall between 0 and the bound of 0.6.
    x = [random.uniform(0, x_bound) for _ in range(N)]
    y = [random.uniform(0, y_bound) for _ in range(N)]
    z = [random.uniform(0, z_bound) for _ in range(N)]
    topology = np.array([x, y, z])
    
    return topology

create_neuron_topology(17000, [600, 600, 600])

array([[455.70692355, 334.92891014, 280.23796058, ..., 256.3507042 ,
        372.50879762, 572.59773561],
       [108.66581544, 137.85567682, 249.45966897, ..., 273.96656499,
        175.65770739, 117.70616517],
       [525.65881218, 372.44661703, 333.61458018, ..., 303.34186194,
        113.27076423, 438.66623063]])

In [10]:
# Creates the group of excitatory neurons.
def create_group_py(topology, noise, masks, group_name='exc_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks

    # Initializing a group of excitatory neurons which contains as many neurons as there are total neurons in the 3D space.
    # It has the following parameters:
    # - py_eqs => Differential equations that define the behavior of the excitatory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - reset => Resets neuron states after they fire according to 'reset_eqs'.
    # - refractory => Sets a refractory period during which an excitatory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_exc = NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -100 mV.
    G_exc.v = '-60*mvolt-rand()*40*mvolt'

    # Sets the neurotransmitter to be used to be glutamate (which is an excitatory neurotransmitter).
    G_exc.glu = 1

    # Assigns the positions of the neurons in micrometers.
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_exc.taille = taille_exc_normale

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    # Applies the treatment and the stimulus masks to the neuron.
    G_exc.treatment_mask = treatment_mask
    G_exc.stimulus_mask = stimulus_mask

    return G_exc

In [11]:
# Creates the group of inhibitory neurons.
def create_group_inh(topology, noise, treatment_mask, group_name='inh_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise

    # Initializing a group of inhibitory neurons which contains as many neurons as there are total neurons in the 3D space.
    # It has the following parameters:
    # - inh_eqs => Differential equations that define the behavior of the inhibitory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - refractory => Sets a refractory period during which an inhibitory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_inh = NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -70 mV.
    G_inh.v = -60*mvolt-rand()*10*mvolt

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_inh.taille = taille_inh_normale

    # Assigns the positions of the neurons in micrometers.
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    # Applies the treatment mask to the neuron.
    G_inh.treatment_mask = treatment_mask

    return G_inh

In [12]:
# Creates a group for Local Field Potential (LFP) recording.
def create_group_lfp(bounds):

    # Extracting the passed on parameters.
    x_bound, y_bound, z_bound = bounds
    
    # Setting up a singular LFP electrode
    Ne = 1

    # Setting the resistivity of the extracellular field to 0.3 siemens per meter, which is within the typical range for biological tissue (0.3-0.4 S/m).
    sigma = 0.3*siemens/meter

    # Initializes a group of neurons, which will only consist of 1 representing a single LFP electrode which has three state variables:
    # - v : volt => Represents the voltage (LFP signal).
    # - x : meter => Represent the x-coordinate of the electrode.
    # - y : meter => Represent the y-coordinate of the electrode.
    # - z : meter => Represent the z-coordinate of the electrode.
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')

    # Setting the x, y, and z coordinates of the LFP electrode to the center of the bounds provided. This places the electrode at the midpoint of the defined 3D space in each dimension.
    lfp.x = x_bound/2*um
    lfp.y = y_bound/2*um
    lfp.z = z_bound/2*um

    return lfp

<br></br>

<h2 style="font-size: 40px;">Network Configuration Function</h2>

In [18]:
# Prepares the network of neurons. The topology is already there, but the neurons still need to be connected, which is done here.
def prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables):

    # Extracting the passed on parameters, including the following parameters:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    p_e2e, p_e2i, p_i2e, p_i2i = current_variables['p']
    topology_exc, topology_inh = topologies
    treatment_mask_exc, treatment_mask_inh = treatment_masks

    
    ##########################################
    ########  Creating Neuron Groups  ########
    ##########################################
    # Creating excitatory and inhibitory neuron groups.
    G_exc = create_group_py(topology_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc])
    G_inh = create_group_inh(topology_inh, current_variables['noise_inh'], treatment_mask_inh)

    # Setting up a Local Field Potential (LFP) electrode.
    G_lfp = create_group_lfp(current_variables['bounds'])
    neuron_groups = [G_exc, G_exc, G_inh, G_lfp]

    
    #####################################
    ########  Creating Monitors  ########
    #####################################
    # Setting up monitors that monitor the population firing rate of excitatory and inhibitory neurons.
    popmon_exc = PopulationRateMonitor(G_exc)
    popmon_inh = PopulationRateMonitor(G_inh)

    # Setting up a monitor that monitors the voltage of the LFP group.
    Mlfp = StateMonitor(G_lfp, 'v', record=True)

    # Setting up monitors that monitor the membrane potential, the noise current (and the stimulus current) for the selected excitatory/inhibitory neurons.
    statemon_exc = StateMonitor(G_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_inh = StateMonitor(G_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    monitors = [popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp]


    ############################################
    ########  Connecting Neuron Groups  ########
    ############################################
    # Configuring synaptic connections between neuron groups where in the function call 'Synapses()' the first parameter is the pre-synaptic group and the second parameter is the post-synaptic group. 
    # The third parameter 'on_pre' specifies the action to be taken when a pre-synaptic neuron fires where the synaptic event will increase the post-synaptic event by a certain amount defined by:
    # - gain => A scaling factor for the synaptic strength.
    # - g_max/siemens => The maximum conductance for the type of synapse normalized by dividing by 'siemens' which is a unit of conductance.
    # - glu_pre => The amount of glutamate released from the pre-synaptic neuron upon firing.
    # MIND: This does not create synapses but instead only specifies their dynamics. The actual synapse connections are created by using the function 'connect()'.
    S_e2e = Synapses(G_exc, G_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    S_e2i = Synapses(G_exc, G_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_e2i')
    S_i2e = Synapses(G_inh, G_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_i2e')

    # Generating the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability.
    # This is done in the following way:
    # - p_e2e => Base probability of connection between neurons of particular groups. 
    # - distance^2 => Squared Euclidean distance between the pre-synaptic and post-synaptic neurons that decreases the overall probability when it is larger.
    # - umeter^2 => Controls the steepness of the exponential decay of the probability.
    S_e2e.connect(p=f'{p_e2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_e2i.connect(p=f'{p_e2i}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_i2e.connect(p=f'{p_i2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({215}*umeter)**2)))')

    # Configuring synaptic connections between the excitatory neurons and the LFP electrode by also specifies a model used for LFP calculation.
    S_lfp = Synapses(G_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')

    # Ensuring LFP voltage is updated after neuron groups.
    S_lfp.summed_updaters['v_post'].when = 'after_groups' 

    # Generating the connections between the excitatory neurons and the LFP electrode. Here, all of them will be connected since there is no probability defined.
    S_lfp.connect()

    # Setting the weight for LFP calculation which is scaled by the Euclidean distance measure.
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'

    synapses = [S_e2e, S_e2i, S_i2e, S_lfp]


    ############################################
    ########  Final Network Definition  ########
    ############################################
    net = Network(neuron_groups, synapses, monitors)

    return net, synapses, monitors

<br></br>

<h2 style="font-size: 40px;">Simulation Functions</h2>

In [19]:
# Runs the simulation of the neural network in discrete time fragments which includes a mechanism to alter neuron parameters based on a firing rate threshold, which simulates a treatment effect.
def run_granular_simulation(net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()

    # Extracting the passed on parameters.
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors

    
    # Setting the potassium equilibrium potentials for both the excitatory and inhibitory neurons.
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()

    # Converting the time fragments to milliseconds and determining the number of batches.
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)

    # For every batch, we run the simulation on the network 'net'. Here the 'tqdm()' function is used which creates a progress bar.
    for i in tqdm(range(num_batches), desc="Running Simulation"): 
        net.run(time_fragment)

        # If after running a time fragment, the average firing rate of the excitatory neurons over the last time fragment exceeds the threshold, treatment is initiated.
        # This statement refers to the scenario where epilepsy is detected and treated by adjusting the potassium equilibrium potentials of the excitatory and inhibitory neurons.
        if np.mean(popmon_exc.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

In [20]:
# This function manages the overall process of setting up and running multiple instances of a neural network simulation as after setting everything up, the function 'run_granular_simulation()' is called.
# In addition, it also handles the storing of the results.
def run_model_loop(variables):

    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')

    # Adding the coordinates of the electrode to the variables array.
    variables["coord_of_electrode"] = populate_electrode_positions(variables)

    # For every run provided in the 'run_id' field of the variables dictionary we perform the simulation (which requires some setting up first).
    for i in range(len(variables['run_id'])):

        ##########################################
        ########  Basic Simulation Setup  ########
        ##########################################
        # Resetting the state of the simulation environment (to avoid interference from the previous run) and seeting the default simulation time step.
        start_scope()
        defaultclock.dt = 0.001*second   

        # Extracting the list of variables required for the current run.
        current_variables = {key: variables[key][i] for key in variables}
        print(current_variables['noise_exc'])

        # Retrieving the treatment settings.
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]

        # Creating a folder to store the results.
        run_id = current_variables["run_id"]
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)

        
        ############################################
        ########  Creating Dynamic Objects  ########
        ############################################
        # Creating the topologies of both the excitatory and inhibitory neurons.
        topology_exc = create_neuron_topology(current_variables['N'][0], current_variables['bounds'])
        topology_inh = create_neuron_topology(current_variables['N'][1], current_variables['bounds'])
        topologies = [topology_exc, topology_inh]

        # Creating the stimulus mask.
        stimulus_geometry_settings = [current_variables['coord_of_stimulus'], current_variables['radius_of_stimulus']]
        stimulus_mask_exc = create_spherical_mask(topology_exc, stimulus_geometry_settings)

        # Creating the treatment mask where all the neurons are included.
        treatment_geometry_settings = [current_variables['coord_of_electrode'], current_variables['radius_of_electrode']]
        treatment_mask_exc = np.ones(current_variables['N'][0])
        treatment_mask_inh = np.ones(current_variables['N'][1])

        # Creating the treatment mask where only a sphere of neurons in the middle of the area is included.
        #treatment_mask_exc = create_spherical_mask(topology_exc, treatment_geometry_settings)
        #treatment_mask_inh = create_spherical_mask(topology_inh, treatment_geometry_settings)
        
        treatment_masks = [treatment_mask_exc, treatment_mask_inh]
        
        # Instantiating the network and setting up the monitors.
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
        
        # Writing the network statistics to a file.
        write_network_statistics(synapses, current_variables['N'], run_id)

        # Plotting and saving the neuron stimulus and treatment mask.
        plot_neuron_masks(topology_exc, [stimulus_mask_exc, treatment_mask_exc], run_id)
        plot_neuron_mask(topology_exc, stimulus_mask_exc, 'red', 'stimulus', run_id)
        plot_neuron_mask(topology_exc, treatment_mask_exc, 'blue', 'treatment', run_id)

        try:
            # Running the simulation with the dynamic objects created above.
            run_granular_simulation(net, current_variables, treatment_settings, monitors)

            # Saving the firing rate data.
            np.savetxt(f'./results/{run_id}/fr_exc.txt', popmon_exc.rate)
            np.savetxt(f'./results/{run_id}/fr_inh.txt', popmon_inh.rate)

            # Plotting the firing rate data together with the noise for both the excitatory and inhibitory neurons.
            plt.plot(popmon_inh.t, popmon_inh.rate, label='Inhibitory')
            plt.plot(popmon_exc.t, popmon_exc.rate, label='Excitatory')

            # Plotting and saving the firing rates.
            plt.legend()
            plt.savefig(f'./results/{run_id}/firing-rates.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the inhibitory neurons.
            plt.plot(statemon_inh.t, statemon_inh.I_noise[4]/nA, label='inh')
            plt.savefig(f'./results/{run_id}/noise_inh.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the excitatory neurons.
            plt.plot(statemon_exc.t, statemon_exc.I_noise[4]/nA, label='exc')
            plt.savefig(f'./results/{run_id}/noise_exc.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the recorded LFP voltage values in millivolts from the 'Mlfp' monitor. 
            np.savetxt(f'./results/{run_id}/LFP.txt', Mlfp.v[0]/mV)
            plot(Mlfp.t/ms, Mlfp.v[0]/mV)
            plt.savefig(f'./results/{run_id}/LFP.png', bbox_inches='tight')
            plt.close()

            
        except:
            print('Broken run.')
        
        time.sleep(1)

<br></br>

<h2 style="font-size: 40px;">Simulation Variables</h2>

In [21]:
########################################
########  Simulation Variables  ########
########################################

# Setting the number of times the simulation will be performed.
copy_times = 5

# Defining the vocabulary of variables.
variables = {
    
    # Defining the simulation settings.
    "run_id": ['Results 1', 'Results 2', 'Results 3', 'Results 4', 'Results 5'],
    "duration": [4000*ms]*copy_times,
    "bounds": [[600, 600, 600]]*copy_times,
    
    # Defining the network settings.
    "N": [[13500, 3375]]*copy_times, # [N_exc, N_inh]

    # Defining the potassium equilibrium potential for both the excitatory and inhibitory neurons.
    # - Healthy mode: Eke_baseline = -90mV
    # - Epileptic mode: Eke_baseline = -84mV
    "Eke_baseline": [-84*mV]*copy_times, 
    "Eki_baseline": [-90*mV]*copy_times,

    # Defining the noise affecting the excitatory and inhibitory neurons.
    "noise_exc": [[0.07, 0.075]*nA]*copy_times, # OLD: [0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Defining the base probabilities of connections between neurons:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500.
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, 

    # Defining the stimulus settings where with the coordinates of (300, 300, 300) the stimulus will be applied to the middle of the neuron block.
    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, 
    "coord_of_stimulus": [[300, 300, 300]]*copy_times,
    "radius_of_stimulus": [180]*copy_times,

    # Defining the treatment settings.
    "device_sensitivity": [8*ms]*copy_times, # Device sensitivity - how frequently to check is firing rate is above the threshold
    "firing_rate_threshold": [5*Hz]*copy_times, 
    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,
    "radius_of_electrode": [200]*copy_times,
    "distance_between_masks": [100]*copy_times,
}

<br></br>

<h2 style="font-size: 40px;">Running the Simulation</h2>

In [22]:
# Running the simulation with the variables.
run_model_loop(variables)


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

WARNING    'glu' is an internal variable of group 'exc_group', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    inh_group's variable 'n' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'v' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'm' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'h' has NaN, very large values, or encountered an error in 

Broken run.
[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

...

Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

In [14]:
run_model_loop(variables)


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100

[70. 75.] pA


FileNotFoundError: [WinError 3] The system cannot find the path specified: './results/Results 1'